In [54]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import torch.nn.init

In [55]:
import visdom

In [56]:
vis = visdom.Visdom()
vis.close(env = "main")

Setting up a new session...


''

In [57]:
def loss_tracker(loss_plot, loss_value, num):
    vis.line(
        Y=torch.Tensor([loss_value]),
        X=torch.Tensor([num]),
        win=loss_plot,
        update='append'
    )


In [58]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


print(device)

cuda


In [59]:
rate=0.001
epoch=15
size = 32

In [60]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [47]:
train_dataloader = torch.utils.data.DataLoader(
                                    dataset=mnist_train,
                                    batch_size=size,
                                    shuffle=True,
                                    drop_last=True)

test_dataloader = torch.utils.data.DataLoader(
                                    dataset=mnist_train,
                                    batch_size=size,
                                    shuffle=False,
                                    drop_last=True)

In [48]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.linear1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.linear2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.linear3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.linear1(x)
        out = self.linear2(out)
        out = self.linear3(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [53]:
## 데이터 전처리 클래스
class CustomDataset(nn.Module):
    def __init__(self, data, transform=None):
        self.images = data['images']
        self.labels = data['labels']
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]

        # Apply the specified transformations to the image
        if self.transform:
            image = self.transform(image)
        image = image.flatten()
        # Convert image and label to PyTorch Tensor
        image = torch.tensor(image, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.long)

        return image, label


## x 모델 만들기
model=CNN().to(device)
dataset = CustomDataset(train_dataloader)
testset = CustomDataset(test_dataloader)

TypeError: 'DataLoader' object is not subscriptable

In [33]:
model = CNN().to(device)

value = (torch.Tensor(1,1,28,28)).to(device)
print((model(value)).shape)


torch.Size([1, 10])


In [34]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = rate)

In [35]:
loss = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='Loss 변화도', legend=['loss'], showlegend=True))


In [36]:
total = len(train_dataloader)
for e in range(epoch):
    cost = 0
    for x,y in train_dataloader:
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        hy=model(x)
        cost = criterion(hy,y)
        cost.backward()
        optimizer.step()
        cost+=cost / total

    print("cost[{}] :{}".format(e+1, cost))
    loss_tracker(loss, torch.Tensor(cost), e)

print("a")

cost[1] :0.004124307539314032
cost[2] :0.012450212612748146
cost[3] :0.3222811818122864
cost[4] :0.002125527011230588
cost[5] :0.004193825647234917
cost[6] :5.456081999000162e-05
cost[7] :0.029983149841427803
cost[8] :0.001881581381894648
cost[9] :0.006198990158736706
cost[10] :2.705854240048211e-05
cost[11] :0.08025290817022324
cost[12] :0.00173280225135386
cost[13] :0.00012140075705247
cost[14] :0.0010602374095469713
cost[15] :3.03216165775666e-05
a
